## Task 2 - Ionosphere Dataset

For task 2, we were tasked in the creation of a predictor using three different models/approaches, which are shown in the following order: (1) Regression, (2) Support Vector Machine and (3) Random Forest.

The analyzed dataset for this task was the Ionosphere dataset, which describes radar data that is collector by a system that is located in Goose Bay Labrador. The system, consisting of a phased array of 16 high frequency antennas, targeted free electrons in the ionosphere. From these observations, a "good" radar return and "bad" radar return can be recorded, where a "good" return is indicative that the radar return showed evidence of some kind of structure in the ionophere and a "bad" return indicates that the signal passed through the ionosphere.

Hence, for classification, our goal is the creation of a predictor that should perform the following classification:
g for good and b for bad = function(ionosphere features)

In our analysis of each created model, each model is conducted and analyzed using a split of training and testing data. Following the intial construction of all models, a 10-fold cross validation is performed to compare model performance betewen different models/approaches. Lastly, this information, along with the results of a t-test are used to identify the best model.


### Ionosphere Data Set Pre-processing

Prior to model creation, the ionosphere data was pre-processed. Note that upon visual inspection of the data, the second column was found to have no variance (e.g. all values were the same) and therefore does not provide a significant contribution to the data. Hence, the second column was removed from the analysis.

For classification, the label was changed to a binary encoding where "g" was mapped to a 1 value and "b" was mapped to a 0 value. This is necessary to construct a logistic regression model so that the regression model maps to a logical value following the performed classification.

In [ ]:
import pandas as pd

# Naming of all columns
colnames=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15',
          'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29',
          'f30', 'f31', 'f32', 'f33', 'f34', 'label']

ionosphere_df = pd.read_csv('data_files/ionosphere.data', names=colnames, header=None)
ionosphere_df

In [ ]:
# Encode the categories as 1 and 0 (g = 1, b = 0)
ionosphere_df['label'] = ionosphere_df.label.astype('category')
encoding = {'g': 1, 'b': 0}
ionosphere_df.label.replace(encoding, inplace=True)

# Removal of the second column (f2) as all of its values are identical and there is no variance
ionosphere_df.drop(columns=['f2'], inplace=True)

X = ionosphere_df.values[:, :-1]
y = ionosphere_df.values[:, -1]

In [ ]:
from sklearn.preprocessing import StandardScaler

# Perform scaling on feature set data
X_scaled = StandardScaler().fit_transform(X)

**NEED TO REWORD THE FOLLOWING**
An important aspect that should be considered when we have multiple features is that not all the features contribute to the model's performance in the same way. Some features are more important than others. To determine which features are most significant, we can use the measures, F-test and p-value.

**F-test** estimates the degree of linear dependency between two random variables when comparing their variances. (See [The F-test for Linear Regression](http://facweb.cs.depaul.edu/sjost/csc423/documents/f-test-reg.htm) for an explanation how to calculate it.)

**p-value** is another measure that helps us determine the significance of each feature. It is used to validate or reject the **null hypothesis**. The null hypothesis ($H_0$) assumes that there is **no relationship** between a given input feature and the output. In the context of our model, it assumes that $\theta_i=0$ for $0 \leq i \le n$, where $n$ is the number of parameters. The meaning of the obtained p-values is as follows:

In [ ]:
from sklearn.feature_selection import f_regression

F_test,p_value = f_regression(X_scaled,y)
pd.DataFrame({'F_test': F_test,'p_value':p_value})

In [ ]:
from sklearn.feature_selection import SelectKBest
X_best = SelectKBest(f_regression).fit_transform(X_scaled, y)

In [ ]:
X_scaled.shape, X_best.shape

<b>(1) Logistic Regression Model</b>

The logistic regression model was evaluated for both scaled and unscaled feature data.

In [ ]:
from sklearn import linear_model
from sklearn import preprocessing

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

# Initialize cross validation for k-fold = 10
cross_validation = KFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
logreg_model = linear_model.LogisticRegression(solver='lbfgs')

In [ ]:
# Non-scaled data
logreg_scores = cross_val_score(logreg_model, X, y, scoring='accuracy', cv=cross_validation, n_jobs=-1)
logreg_scores

In [ ]:
# Scaled data
logreg_scores = cross_val_score(logreg_model, X_scaled, y, scoring='accuracy', cv=cross_validation, n_jobs=-1)
logreg_scores

In [ ]:
# Selection of k-best
logreg_scores = cross_val_score(logreg_model, X_best, y, scoring='accuracy', cv=cross_validation, n_jobs=-1)
logreg_scores

<b>(2) Support Vector Machine</b>

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.svm import SVC

svc_soft_model = SVC(kernel='linear')
svc_soft_model.fit(X_scaled, y)

In [ ]:
Yp_soft = svc_soft_model.predict(X_scaled)

In [ ]:
from sklearn.metrics import accuracy_score
print("Soft Margin Model accuracy:",accuracy_score(y,Yp_soft))

In [ ]:
# K-fold cross validation
svc_soft_model_scores = cross_val_score(model_soft, X_scaled, y, scoring='accuracy', cv=cross_validation, n_jobs=-1)
svc_soft_model_scores

<b>(3) Random Forest</b>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=12)
random_forest_model.fit(X_scaled, y)

In [ ]:
# K-fold cross validation
random_forest_model_scores = cross_val_score(model_soft, X_scaled, y, scoring='accuracy', cv=cross_validation, n_jobs=-1)
random_forest_model_scores

<b>Model Performance Comparison</b>

In [ ]:
#t-test, comparison of model performance